# co-visitation

In [1]:
VER = 5

import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import cudf, itertools
print('We will use RAPIDS version',cudf.__version__)

We will use RAPIDS version 25.02.00


In [2]:
%%time
def read_file(f):
    return cudf.DataFrame( data_cache[f] )
def read_file_to_cache(f):
    df = pd.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(type_labels).astype('int8')
    return df
    
data_cache = {}
type_labels = {'clicks':0, 'carts':1, 'orders':2}
files = glob.glob('/kaggle/input/otto-chunk-data-inparquet-format/test_parquet/*')+glob.glob('/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/*')
for f in files: data_cache[f] = read_file_to_cache(f)

READ_CT = 5
CHUNK = int( np.ceil( len(files)/6 ))
print(f'We will process {len(files)} files, in groups of {READ_CT} and chunks of {CHUNK}.')

We will process 146 files, in groups of 5 and chunks of 25.
CPU times: user 43.6 s, sys: 9.16 s, total: 52.8 s
Wall time: 47.8 s


In [3]:
%%time
type_weight = {0:1, 1:6, 2:3}

DISK_PIECES = 4
SIZE = 1.86e6/DISK_PIECES

for PART in range(DISK_PIECES):
    print()
    print('### DISK PART',PART+1)
    
    for j in range(6):
        a = j*CHUNK
        b = min( (j+1)*CHUNK, len(files) )
        print(f'Processing files {a} thru {b-1} in groups of {READ_CT}...')
        
        for k in range(a,b,READ_CT):
            # READ FILE
            df = [read_file(files[k])]
            for i in range(1,READ_CT): 
                if k+i<b: df.append( read_file(files[k+i]) )
            df = cudf.concat(df,ignore_index=True,axis=0)
            df = df.sort_values(['session','ts'],ascending=[True,False])
            df = df.reset_index(drop=True)
            df['n'] = df.groupby('session').cumcount()
            df = df.loc[df.n<30].drop('n',axis=1)
            df = df.merge(df,on='session')
            df = df.loc[ ((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y) ]
            df = df.loc[(df.aid_x >= PART*SIZE)&(df.aid_x < (PART+1)*SIZE)]
            df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
            df['wgt'] = df.type_y.map(type_weight)
            df = df[['aid_x','aid_y','wgt']]
            df.wgt = df.wgt.astype('float32')
            df = df.groupby(['aid_x','aid_y']).wgt.sum()
            if k==a: tmp2 = df
            else: tmp2 = tmp2.add(df, fill_value=0)
            print(k,', ',end='')
        print()
        if a==0: tmp = tmp2
        else: tmp = tmp.add(tmp2, fill_value=0)
        del tmp2, df
        gc.collect()
    tmp = tmp.reset_index()
    tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
    tmp = tmp.reset_index(drop=True)
    tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
    tmp = tmp.loc[tmp.n<40].drop('n',axis=1)
    tmp.to_pandas().to_parquet(f'top_40_carts_orders_v{VER}_{PART}.pqt')


### DISK PART 1
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5...
25 , 30 , 35 , 40 , 45 , 
Processing files 50 thru 74 in groups of 5...
50 , 55 , 60 , 65 , 70 , 
Processing files 75 thru 99 in groups of 5...
75 , 80 , 85 , 90 , 95 , 
Processing files 100 thru 124 in groups of 5...
100 , 105 , 110 , 115 , 120 , 
Processing files 125 thru 145 in groups of 5...
125 , 130 , 135 , 140 , 145 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5...
25 , 30 , 35 , 40 , 45 , 
Processing files 50 thru 74 in groups of 5...
50 , 55 , 60 , 65 , 70 , 
Processing files 75 thru 99 in groups of 5...
75 , 80 , 85 , 90 , 95 , 
Processing files 100 thru 124 in groups of 5...
100 , 105 , 110 , 115 , 120 , 
Processing files 125 thru 145 in groups of 5...
125 , 130 , 135 , 140 , 145 , 

### DISK PART 3
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 

In [4]:
%%time
DISK_PIECES = 4
SIZE = 1.86e6/DISK_PIECES

for PART in range(DISK_PIECES):
    print()
    print('### DISK PART',PART+1)
    
    for j in range(6):
        a = j*CHUNK
        b = min( (j+1)*CHUNK, len(files) )
        print(f'Processing files {a} thru {b-1} in groups of {READ_CT}...')
        
        for k in range(a,b,READ_CT):
            # READ FILE
            df = [read_file(files[k])]
            for i in range(1,READ_CT): 
                if k+i<b: df.append( read_file(files[k+i]) )
            df = cudf.concat(df,ignore_index=True,axis=0)
            df = df.loc[df['type'].isin([1,2])]
            df = df.sort_values(['session','ts'],ascending=[True,False])
            df = df.reset_index(drop=True)
            df['n'] = df.groupby('session').cumcount()
            df = df.loc[df.n<30].drop('n',axis=1)
            df = df.merge(df,on='session')
            df = df.loc[ ((df.ts_x - df.ts_y).abs()< 14 * 24 * 60 * 60) & (df.aid_x != df.aid_y) ]
            df = df.loc[(df.aid_x >= PART*SIZE)&(df.aid_x < (PART+1)*SIZE)]
            df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
            df['wgt'] = 1
            df = df[['aid_x','aid_y','wgt']]
            df.wgt = df.wgt.astype('float32')
            df = df.groupby(['aid_x','aid_y']).wgt.sum()
            if k==a: tmp2 = df
            else: tmp2 = tmp2.add(df, fill_value=0)
            print(k,', ',end='')
        print()
        if a==0: tmp = tmp2
        else: tmp = tmp.add(tmp2, fill_value=0)
        del tmp2, df
        gc.collect()
    tmp = tmp.reset_index()
    tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
    tmp = tmp.reset_index(drop=True)
    tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
    tmp = tmp.loc[tmp.n<40].drop('n',axis=1)
    tmp.to_pandas().to_parquet(f'top_40_buy2buy_v{VER}_{PART}.pqt')


### DISK PART 1
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5...
25 , 30 , 35 , 40 , 45 , 
Processing files 50 thru 74 in groups of 5...
50 , 55 , 60 , 65 , 70 , 
Processing files 75 thru 99 in groups of 5...
75 , 80 , 85 , 90 , 95 , 
Processing files 100 thru 124 in groups of 5...
100 , 105 , 110 , 115 , 120 , 
Processing files 125 thru 145 in groups of 5...
125 , 130 , 135 , 140 , 145 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5...
25 , 30 , 35 , 40 , 45 , 
Processing files 50 thru 74 in groups of 5...
50 , 55 , 60 , 65 , 70 , 
Processing files 75 thru 99 in groups of 5...
75 , 80 , 85 , 90 , 95 , 
Processing files 100 thru 124 in groups of 5...
100 , 105 , 110 , 115 , 120 , 
Processing files 125 thru 145 in groups of 5...
125 , 130 , 135 , 140 , 145 , 

### DISK PART 3
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 

In [5]:
%%time
DISK_PIECES = 4
SIZE = 1.86e6/DISK_PIECES

for PART in range(DISK_PIECES):
    print()
    print('### DISK PART',PART+1)
    
    for j in range(6):
        a = j*CHUNK
        b = min( (j+1)*CHUNK, len(files) )
        print(f'Processing files {a} thru {b-1} in groups of {READ_CT}...')
        
        for k in range(a,b,READ_CT):
            df = [read_file(files[k])]
            for i in range(1,READ_CT): 
                if k+i<b: df.append( read_file(files[k+i]) )
            df = cudf.concat(df,ignore_index=True,axis=0)
            df = df.sort_values(['session','ts'],ascending=[True,False])
            df = df.reset_index(drop=True)
            df['n'] = df.groupby('session').cumcount()
            df = df.loc[df.n<30].drop('n',axis=1)
            df = df.merge(df,on='session')
            df = df.loc[ ((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y) ]
            df = df.loc[(df.aid_x >= PART*SIZE)&(df.aid_x < (PART+1)*SIZE)]
            df = df[['session', 'aid_x', 'aid_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
            df['wgt'] = 1 + 3*(df.ts_x - 1659304800)/(1662328791-1659304800)
            df = df[['aid_x','aid_y','wgt']]
            df.wgt = df.wgt.astype('float32')
            df = df.groupby(['aid_x','aid_y']).wgt.sum()
            if k==a: tmp2 = df
            else: tmp2 = tmp2.add(df, fill_value=0)
            print(k,', ',end='')
        print()
        if a==0: tmp = tmp2
        else: tmp = tmp.add(tmp2, fill_value=0)
        del tmp2, df
        gc.collect()
    tmp = tmp.reset_index()
    tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
    tmp = tmp.reset_index(drop=True)
    tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
    tmp = tmp.loc[tmp.n<45].drop('n',axis=1)
    tmp.to_pandas().to_parquet(f'top_45_clicks_v{VER}_{PART}.pqt')


### DISK PART 1
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5...
25 , 30 , 35 , 40 , 45 , 
Processing files 50 thru 74 in groups of 5...
50 , 55 , 60 , 65 , 70 , 
Processing files 75 thru 99 in groups of 5...
75 , 80 , 85 , 90 , 95 , 
Processing files 100 thru 124 in groups of 5...
100 , 105 , 110 , 115 , 120 , 
Processing files 125 thru 145 in groups of 5...
125 , 130 , 135 , 140 , 145 , 

### DISK PART 2
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5...
25 , 30 , 35 , 40 , 45 , 
Processing files 50 thru 74 in groups of 5...
50 , 55 , 60 , 65 , 70 , 
Processing files 75 thru 99 in groups of 5...
75 , 80 , 85 , 90 , 95 , 
Processing files 100 thru 124 in groups of 5...
100 , 105 , 110 , 115 , 120 , 
Processing files 125 thru 145 in groups of 5...
125 , 130 , 135 , 140 , 145 , 

### DISK PART 3
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 

In [6]:
# FREE MEMORY
del data_cache, tmp
_ = gc.collect()

# generate candidates

In [7]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}
def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob('/kaggle/input/otto-chunk-data-inparquet-format/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

test_df = load_test()
print('Test data has shape',test_df.shape)
test_df.head()

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Test data has shape (6928123, 4)


,session,aid,ts,type
0,13099779,245308,1661795832,0
1,13099779,245308,1661795862,1
2,13099779,972319,1661795888,0
3,13099779,972319,1661795898,1
4,13099779,245308,1661795907,0


In [8]:
def load_full_train():    
    dfs = []
    files_a = glob.glob('/kaggle/input/otto-chunk-data-inparquet-format/test_parquet/*')
    files_b = glob.glob('/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/*')
    all_files = files_a + files_b
    for e, chunk_file in enumerate(all_files):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

full_train_df = load_full_train()
print('full_train data has shape',full_train_df.shape)

full_train data has shape (223644219, 4)


In [9]:
top_clicks = test_df.loc[test_df['type']==0,'aid'].value_counts().index.values[:20]
top_orders = test_df.loc[test_df['type']==2,'aid'].value_counts().index.values[:20]
full_top_orders = full_train_df.loc[full_train_df['type']==2,'aid'].value_counts().index.values[:200]

In [14]:
%%time
DISK_PIECES = 4
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()
# LOAD THREE CO-VISITATION MATRICES
top_20_clicks = pqt_to_dict( pd.read_parquet(f'top_45_clicks_v5_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(f'top_45_clicks_v{VER}_{k}.pqt') ) )
    
top_20_buys = pqt_to_dict( pd.read_parquet(f'top_40_carts_orders_v{VER}_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_20_buys.update( pqt_to_dict( pd.read_parquet(f'top_40_carts_orders_v{VER}_{k}.pqt') ) )
    
top_20_buy2buy = pqt_to_dict( pd.read_parquet(f'top_40_buy2buy_v{VER}_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_20_buy2buy.update( pqt_to_dict( pd.read_parquet(f'top_40_buy2buy_v{VER}_{k}.pqt') ) )



print('Here are size of our 3 co-visitation matrices:')
print( len( top_20_clicks ), len( top_20_buy2buy ), len( top_20_buys ) )


Here are size of our 3 co-visitation matrices:
1837166 1168768 1837166
CPU times: user 1min 57s, sys: 10.3 s, total: 2min 7s
Wall time: 1min 55s


In [15]:
type_weight_multipliers = {0: 1, 1: 6, 2: 3}

def suggest_clicks(df):
    aids=df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    if len(unique_aids)>=50:
        weights=np.logspace(0.1,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        sorted_aids = [k for k,v in aids_temp.most_common(50)]
        return sorted_aids
    aids2 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2).most_common(50) if aid2 not in unique_aids]    
    result = unique_aids + top_aids2[:50 - len(unique_aids)]
    return result + list(top_clicks)[:50-len(result)]

def suggest_buys(df):
    aids=df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    df = df.loc[(df['type']==1)|(df['type']==2)]
    unique_buys = list(dict.fromkeys( df.aid.tolist()[::-1] ))
    if len(unique_aids)>=50:
        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
        for aid in aids3: aids_temp[aid] += 0.1
        sorted_aids = [k for k,v in aids_temp.most_common(50)]
        return sorted_aids
    aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2+aids3).most_common(50) if aid2 not in unique_aids] 
    result = unique_aids + top_aids2[:50 - len(unique_aids)]
    return result + list(top_orders)[:50-len(result)]

def suggest_candidates_50(df):
    cand_clicks = suggest_clicks(df)
    cand_buys   = suggest_buys(df)

    merged = cand_clicks + cand_buys
    merged_unique = list(dict.fromkeys(merged))

    if len(merged_unique) < 50:
        needed = 50 - len(merged_unique)
        fill_items = []
        for item in full_top_orders:
            if item not in merged_unique:
                fill_items.append(item)
                if len(fill_items) >= needed:
                    break
        merged_unique += fill_items

    return merged_unique[:50]

In [16]:
candidate_rows = []
for session_id, subdf in test_df.groupby('session', sort=False):
    cands_50 = suggest_candidates_50(subdf)
    for aid in cands_50:
        candidate_rows.append((session_id, aid))

candidate_df = pd.DataFrame(candidate_rows, columns=['session','aid'])
print(candidate_df.shape)

(83590150, 2)


In [17]:
pd.set_option('display.max_rows', None)
print(candidate_df.head(200))

      session      aid
0    13099779   245308
1    13099779   972319
2    13099779  1100749
3    13099779  1090801
4    13099779   717801
5    13099779   885958
6    13099779  1734314
7    13099779   352868
8    13099779   128649
9    13099779   595204
10   13099779   319580
11   13099779   116651
12   13099779  1552337
13   13099779  1575802
14   13099779   441353
15   13099779   745721
16   13099779   428539
17   13099779  1332889
18   13099779  1801838
19   13099779  1303193
20   13099779   587671
21   13099779   908814
22   13099779  1491863
23   13099779   798152
24   13099779  1627951
25   13099779  1732664
26   13099779  1132027
27   13099779  1534310
28   13099779   942224
29   13099779  1311574
30   13099779  1270528
31   13099779  1633488
32   13099779   803272
33   13099779   970197
34   13099779   240614
35   13099779  1481491
36   13099779   224593
37   13099779   727122
38   13099779  1482503
39   13099779  1082306
40   13099779  1710382
41   13099779   843086
42   130997

In [18]:
candidate_df.to_parquet("candidates.parquet", index=False)